In [ ]:
import pandas as pd
import pvlib
from datetime import datetime, timedelta, timezone
import pytz


In [ ]:
def calc_power_output(latitude, longitude, rated_power_per_panel, number_of_panels, panel_tilt, panel_azimuth):    
    location = pvlib.location.Location(latitude, longitude)
    #clearsky_irradiance =  location.get_clearsky(times)

    # Typical meteorogical year using the pvgis api
    data, _, _, _ = pvlib.iotools.get_pvgis_tmy(latitude=latitude, longitude=longitude)

    solar_position = location.get_solarposition(data.index)

    #solar_position = location.get_solarposition(date_time)
    # Optimisic estimate of solar irradiation because it assumes clear skies
    # clear_sky = location.get_clearsky(date_time, model="ineichen")

    # Calculates the irradiance watts per m squared of the panel surface
    # Pvlib assumes a gorund albedo of around .2 which is correspondant with grass and soil, may differ for highly reflective surfaces like snow
    irradiance = pvlib.irradiance.get_total_irradiance(
        surface_tilt=panel_tilt,
        surface_azimuth=panel_azimuth,
        solar_zenith=solar_position['zenith'],
        solar_azimuth=solar_position['azimuth'],
        ghi=data['ghi'],
        dni=data['dni'],
        dhi=data['dhi'])
    
    #default values entered for wind_speed, u0, and u1
    module_temperature = pvlib.temperature.faiman(poa_global=irradiance["poa_global"], temp_air=data["temp_air"], wind_speed=1, u0=25.0, u1=6.84)

    dc_power = pvlib.pvsystem.pvwatts_dc(
    g_poa_effective=irradiance['poa_global'],
    # simplified assumptions that module and cell temperature are the same
    temp_cell=module_temperature,
    pdc0=rated_power_per_panel,
    gamma_pdc=-0.004,
    temp_ref=25.0,
    )
    #dc_power.index = dc_power.index.replace(year=2024)
    dc_power.index = dc_power.index.map(lambda x: x.replace(year=2024))

    # Plot x-axis: time, y-axis: dc_power on a line chart thingy
    #matplotlib.pyplot.plot
    

    return dc_power * number_of_panels



def rounds_and_calculates_a_year_of_dates():
    
    tz = "Europe/Dublin"
    #times = pd.date_range(start='2025-03-29 00:00:00', end='2025-03-30 00:00:00', freq='30min', tz='Europe/Dublin')
    tz_formatted = pytz.timezone(tz)
    time_now_aware = datetime.now(tz_formatted)
    delta = timedelta(minutes=30)
    time_rounded_down = time_now_aware - (time_now_aware - datetime.min.replace(tzinfo=timezone.utc)) % delta
    return pd.date_range(start=time_rounded_down.replace(year=(int(time_rounded_down.year) - 1)), end=time_rounded_down, freq="30min", tz=tz)

In [200]:
times_from_now = rounds_and_calculates_a_year_of_dates()

clearsky_irradiance = calc_power_output(55.6846, 12.5389, 6, 0.18, 40, 180, times_from_now)



#clearsky_irradiance
clearsky_irradiance = clearsky_irradiance.reset_index().rename(columns={"time(UTC)": "x", 0: "y"})

clearsky_irradiance = clearsky_irradiance[(clearsky_irradiance["x"] >= "2024-07-23T00:00:00.000") & (clearsky_irradiance["x"] < "2024-07-24T00:00:00.000")]


clearsky_irradiance = clearsky_irradiance.to_json(orient="records", date_format="iso")

clearsky_irradiance


'[{"x":"2024-07-23T00:00:00.000Z","y":0.0},{"x":"2024-07-23T01:00:00.000Z","y":0.0},{"x":"2024-07-23T02:00:00.000Z","y":0.0},{"x":"2024-07-23T03:00:00.000Z","y":0.0},{"x":"2024-07-23T04:00:00.000Z","y":2.2568816455},{"x":"2024-07-23T05:00:00.000Z","y":11.7753322389},{"x":"2024-07-23T06:00:00.000Z","y":11.4643858707},{"x":"2024-07-23T07:00:00.000Z","y":47.2801902643},{"x":"2024-07-23T08:00:00.000Z","y":133.3286177001},{"x":"2024-07-23T09:00:00.000Z","y":127.2409254919},{"x":"2024-07-23T10:00:00.000Z","y":186.4374227022},{"x":"2024-07-23T11:00:00.000Z","y":156.1378293923},{"x":"2024-07-23T12:00:00.000Z","y":148.5656400622},{"x":"2024-07-23T13:00:00.000Z","y":122.8041897676},{"x":"2024-07-23T14:00:00.000Z","y":152.1402735361},{"x":"2024-07-23T15:00:00.000Z","y":140.0427187037},{"x":"2024-07-23T16:00:00.000Z","y":81.4691232086},{"x":"2024-07-23T17:00:00.000Z","y":40.6768568645},{"x":"2024-07-23T18:00:00.000Z","y":14.773359815},{"x":"2024-07-23T19:00:00.000Z","y":1.6831194566},{"x":"2024-07

In [202]:
import pandas

hash_id = "krwm2n2y"
esb_intake = pandas.read_csv(f"/Users/michealmcmagh/Desktop/ise-entrance-submission-project/solarcalc/uploads/{hash_id}.csv")
esb_intake = pandas.concat([esb_intake["Read Date and End Time"], esb_intake["Read Value"]], join="inner", axis=1)
esb_intake = esb_intake.rename(columns={"Read Date and End Time": "x", "Read Value": "y"})
esb_intake["y"] = esb_intake["y"] * 1000
esb_intake["x"] = pandas.to_datetime(esb_intake["x"], dayfirst=True)
esb_intake = esb_intake.sort_values(by=['x'])
esb_intake = esb_intake[(esb_intake["x"] >= "2024-07-23T00:00:00.000") & (esb_intake["x"] < "2024-07-24T00:00:00.000")]


json_esb_intake = esb_intake.to_json(orient='records', date_format='iso')
json_esb_intake

'[{"x":"2024-07-23T00:00:00.000","y":494.0},{"x":"2024-07-23T00:30:00.000","y":440.0},{"x":"2024-07-23T01:00:00.000","y":598.0},{"x":"2024-07-23T01:30:00.000","y":360.0},{"x":"2024-07-23T02:00:00.000","y":298.0},{"x":"2024-07-23T02:30:00.000","y":290.0},{"x":"2024-07-23T03:00:00.000","y":1316.0},{"x":"2024-07-23T03:30:00.000","y":568.0},{"x":"2024-07-23T04:00:00.000","y":280.0},{"x":"2024-07-23T04:30:00.000","y":276.0},{"x":"2024-07-23T05:00:00.000","y":268.0},{"x":"2024-07-23T05:30:00.000","y":264.0},{"x":"2024-07-23T06:00:00.000","y":276.0},{"x":"2024-07-23T06:30:00.000","y":392.0},{"x":"2024-07-23T07:00:00.000","y":386.0},{"x":"2024-07-23T07:30:00.000","y":362.0},{"x":"2024-07-23T08:00:00.000","y":380.0},{"x":"2024-07-23T08:30:00.000","y":1014.0},{"x":"2024-07-23T09:00:00.000","y":280.0},{"x":"2024-07-23T09:30:00.000","y":264.0},{"x":"2024-07-23T10:00:00.000","y":1284.0},{"x":"2024-07-23T10:30:00.000","y":1086.0},{"x":"2024-07-23T11:00:00.000","y":852.0},{"x":"2024-07-23T11:30:00.00